In [10]:
#from textblob import TextBlob
#from attributegetter import *

from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

In [59]:
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''
    
    
    print("Please confirm if we can book", attributes['NumberofTickets'] ,"tickets for", attributes['Movie'], "movie on", attributes['Showtime'] ,"in theater" , attributes['Theatres'],' ', attributes['location']," Yes or No" )
    context = IntentComplete()
    return 'action: ' + current_intent.action, context

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    for param in current_intent.params:
         if param.required:
            if param.name not in attributes:
                context = param.context
                return random.choice(param.prompts), context
    return None, context

def loadIntent(path, intent):
    #print(path,intent)
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])



In [38]:
def RasaIntentIdentifier(user_input, context, current_intent, interpreter):
    clean_input = user_input.lower()
    #scores = ngrammatch(clean_input)
    #scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    # print clean_input
    #print 'scores', scores
    
    d = interpreter.parse(clean_input)
    current_intent = d["intent"]["name"]
    attributes = {}
    for entity in d["entities"]:
        attributes[entity["entity"]] = entity["value"]
    
    print(attributes)
    if(current_intent == "movie_search"):
        return loadIntent('params/MovieParams2.cfg', 'SearchMovie'), attributes
    else:
        #print 'same intent'
        return current_intent


In [61]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Contexts are flags which control dialogue flow, see Contexts.py
        self.active_contexts = active_contexts
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        model_directory = "models/movie/default/model_20180323-165349"
        self.interpreter = Interpreter.load(model_directory, RasaNLUConfig("config_spacy.json"))

    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        #self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        current_attributes = {}
        self.current_intent,current_attributes = RasaIntentIdentifier(clean_input, self.context, self.current_intent, self.interpreter)
        self.attributes.update(current_attributes)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [62]:
session = Session()

print ('BOT: Hi! How may I assist you?')

while True:
    
    inp = input('User: ')
    if inp == 'exit':
        break
    else:   
        print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you?
User: i'm looking for a movie in the gachibowli
{'location': 'gachibowli', 'language': 'telugu'}


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


AttributeError: 'str' object has no attribute 'name'

In [9]:
d = interpreter.parse("i'm looking for a movie in the gachibowli")
#print(d)
print(d["intent"],d["entities"])

{'name': 'movie_search', 'confidence': 0.7065738492559193} [{'end': 41, 'extractor': 'ner_crf', 'entity': 'location', 'start': 31, 'value': 'gachibowli'}]


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [57]:
temp_context = None
temp_current_intent = FirstGreeting()
clean_input = "i'm looking for a telugu movie in the gachibowli"
temp_attributes = {}
temp_intent,temp_attributes = RasaIntentIdentifier(clean_input, temp_context, temp_current_intent, interpreter)
print(temp_intent.name,temp_intent.params,temp_attributes)

{'location': 'gachibowli', 'language': 'telugu'}
SearchMovie [<Intents.Parameter object at 0x7ff44cdc8a90>, <Intents.Parameter object at 0x7ff44cdc8e80>, <Intents.Parameter object at 0x7ff44cdc8208>] {'location': 'gachibowli', 'language': 'telugu'}


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [60]:
temp_prompt, temp_context = check_required_params(temp_intent, temp_attributes, temp_context)
print(temp_attributes)
print(temp_prompt)
print(temp_context)


{'location': 'gachibowli', 'language': 'telugu'}
Please specify the genre of the movie(eg : Action,Comedy,Family)?
GetGenre


In [53]:
for i in temp_intent.params:
    print(i.name, i.prompts)

location ['In which city you want to search for the movie(eg: Hyderabad or Bangalore)?']
language ['Please enter the language of the movie(eg : Hindi,English,Telugu)?']
genre ['Please specify the genre of the movie(eg : Action,Comedy,Family)?']
